Testbed for bots, by Arseny, based on Henry's testing_grounds.

In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

import draftsimtools as ds

In [2]:
# Load M19 drafts
raw_drafts = ds.load_drafts("../../data/m19_2.csv")

In [100]:
# Here other folks load card lists, but I grab them from json instead
# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

with open('../../data/Allsets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
    
jsonSubset = mtgJSON['M19']['cards']

thisSet = {card['name'] : card for card in jsonSubset}

In [99]:
# thisSet[list(thisSet.keys())[0]] # Just to look at the fields

In [ ]:
# One (simple) way to create a list of names:
# nameList = pd.DataFrame(thisSet.keys(),columns = ['Name']) # Instead of relying on cvs, get names from json

In [114]:
# Another (fancier) way to create a list of names + lots of other useful stuff
nameList = pd.DataFrame.from_dict(thisSet, orient='index', columns=['colors','rarity','type','convertedManaCost'])
nameList['Name'] = a.index                 # We need names as a column, not an index
nameList['index'] = range(len(nameList))
nameList = nameList.set_index('index')     # And we need a normal numerical index
nameList[1:5]

,colors,rarity,type,convertedManaCost,Name
index,,,,,
1,[R],common,Sorcery,3.0,Act of Treason
2,[W],uncommon,Instant,2.0,Aegis of the Heavens
3,"[U, W]",uncommon,Creature — Human Artificer,4.0,Aerial Engineer
4,[U],uncommon,Enchantment — Aura,2.0,Aether Tunnel


In [115]:
# Process names, then handle weird card names (those with commas)

nameList['Name'] = nameList.Name.str.replace(' ','_')

# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both
# nameList.Name[nameList.Name.str.find(',')!=-1] # There should be no longer any cards with commas

In [104]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.
Processing draft: 10000.
Processing draft: 20000.
Processing draft: 30000.
Processing draft: 40000.
Processing draft: 50000.
Processing draft: 60000.
Processing draft: 70000.
Processing draft: 80000.
Processing draft: 90000.
Processing draft: 100000.


### Now this part below will just create weird bots and test them

But for now I can as well sketch a random bot here.

In [105]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

In [106]:
# nameList.Name = nameList.Name.str.lower()
nCardsInSet = len(nameList)
nCardsInSet

300

In [107]:
class bot_random(object):
    def __init__(self,thisSet):
        self.thisSet = thisSet # a list with 'Name' column, containing the names
        
    def pickCard(self,pack,collection):
        cardInd = np.random.choice(np.nonzero(hand)[0])
        return cardInd

In [116]:
bot = bot_random(nameList)

# Count-hot encoding, then colling the bot
pickCount = 0
accuracySimple = 0

for iDraft in range(20): #range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    collection = np.zeros(nCardsInSet+1)
    for pack in draft:     
        hand = np.zeros(nCardsInSet+1)
        iCard = 0
        for card in pack:
            cardName = card
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                pos = nCardsInSet # Use this very large number (300 for M19) if not found; for now
                # print("---Unrecognized card:") # All unrecognized cards here seem to be foil lands, so ignore 'em
            hand[pos] += 1
            
            if card==pack[0]: # First card, the one that was picked
                humanCard = pos
                
        # ----- that's where the bot should be called, with current collection and pack
        botCard = bot.pickCard(hand, collection)
        #print(len(pack),botCard, humanCard==botCard)
        
        pickCount += 1
        if(humanCard==botCard):
            accuracySimple += 1 # Simple count of picked cards
        
        collection[humanCard] += 1 # Update collection
        
print("Simple accuracy: %4.2f" % (accuracySimple/pickCount))

Simple accuracy: 0.21


In [ ]:
# Gets bot accuracy for some packs
tester = ds.BotTester(test)
tester.evaluate_bots([draftsim_bot, sgd_bot], ["Draftsim2018", "Draftsim2018SGD"])
tester.write_evaluations()